In [1]:
import pandas as pd
import numpy as np
#import matplotlib
import matplotlib.pyplot as plt


Bad key "text.kerning_factor" on line 4 in
/Users/eyobmanhardt/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
df = pd.read_csv('temp_preprocess_data.csv')

In [3]:
print('shape:', df.shape)
df.head()

shape: (356251, 131)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_REQ_CREDIT_BUREAU_YEAR,AMT_CREDIT_DEBT_RATIO,CREDIT_DAY_OVERDUE,DPD_COUNTS,BIRTH_IN_YEARS,ALL_EXT_SOURCE_MEAN,PAYMENT_RATE,INCOME_PER_PERSON,INCOME_CREDIT_IN_PERCENTAGE,ANNUITY_INCOME_IN_PERCENTAGE
0,100002,1.0,Cash loans,0,1,0,0,202500.0,406597.5,24700.5,...,1.0,0.068272,0.0,3.375,25.920548,0.161787,0.060749,202500.0,0.498036,0.121978
1,100003,0.0,Cash loans,1,1,1,0,270000.0,1293502.5,35698.5,...,0.0,0.000000,0.0,NaN,45.931507,0.466757,0.027598,135000.0,0.208736,0.132217
2,100004,0.0,Revolving loans,0,0,0,0,67500.0,135000.0,6750.0,...,0.0,0.000000,0.0,NaN,52.180822,0.642739,0.050000,67500.0,0.500000,0.100000
3,100006,0.0,Cash loans,1,1,0,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,52.068493,0.650442,0.094941,67500.0,0.431748,0.219900
4,100007,0.0,Cash loans,0,1,0,0,121500.0,513000.0,21865.5,...,0.0,0.000000,0.0,NaN,54.608219,0.322738,0.042623,121500.0,0.236842,0.179963


In [4]:
train_df = df.loc[df['TARGET'].notnull(), :]
test_df = df.loc[df['TARGET'].isnull(), :]
test_df.drop(columns= ['TARGET'], axis=1, inplace=True)

train_df_enc = pd.get_dummies(train_df)
test_df_enc = pd.get_dummies(test_df)
print('train shape:', train_df_enc.shape) 
print('test shape:', test_df_enc.shape) 

/Users/eyobmanhardt/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


train shape: (307507, 251)
test shape: (48744, 248)


In [5]:
train_df_enc_cols = set(train_df_enc.columns)
test_df_enc_cols = set(test_df_enc.columns)

print(train_df_enc_cols.difference(test_df_enc_cols))

{'NAME_FAMILY_STATUS_Unknown', 'NAME_INCOME_TYPE_Maternity leave', 'TARGET'}


In [6]:
y = train_df_enc['TARGET']
## align dataframes 
X, X_test = train_df_enc.align(test_df_enc, join='inner', axis=1)

In [7]:
X.shape, X_test.shape, y.shape

((307507, 248), (48744, 248), (307507,))

In [8]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='median')

In [9]:
Test_ids = X_test['SK_ID_CURR']
X.drop(columns= ['SK_ID_CURR'], inplace=True)
X_test.drop(columns= ['SK_ID_CURR'], inplace=True)

In [10]:
X.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,0,1,0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,0,0,0,0,0,0,1,0,1,0
1,1,1,1,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,...,0,1,0,0,0,0,0,0,1,0
2,0,0,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,...,0,0,0,0,0,0,0,0,0,0


In [11]:
## fix the inf problem 
X['AMT_CREDIT_DEBT_RATIO'] = X['AMT_CREDIT_DEBT_RATIO'].apply(lambda x: x if ~np.isinf(x) else 0)
X_test['AMT_CREDIT_DEBT_RATIO'] = X_test['AMT_CREDIT_DEBT_RATIO'].apply(lambda x: x if ~np.isinf(x) else 0)

In [12]:
## impute missing values by its median 
imp.fit(X)
X = imp.transform(X)
X_test = imp.fit_transform(X_test)

In [13]:
# create another df for sampling 
diff = train_df_enc_cols.difference(test_df_enc_cols)
cols = [c for c in train_df_enc.columns if c not in diff]

df_prime = pd.DataFrame(X, columns = cols[1:])
# add target
df_prime['TARGET'] = y

# separate classes
df_class0 = df_prime.loc[df_prime['TARGET']==0, :]
df_class1 = df_prime.loc[df_prime['TARGET']==1, :]


# down-sampling

In [14]:
print('Before sampling')
print(df_class0.shape, df_class1.shape)

df_class0_down_sample = df_class0.sample(df_class1.shape[0])

print('After sampling')
print(df_class0_down_sample.shape, df_class1.shape)

# concat the two classes (down sample)
df_ds = pd.concat([df_class0_down_sample, df_class1], axis = 0)
print('new df shape:', df_ds.shape)

Before sampling
(282682, 248) (24825, 248)
After sampling
(24825, 248) (24825, 248)
new df shape: (49650, 248)


In [15]:
df_ds.TARGET.value_counts()

1.0    24825
0.0    24825
Name: TARGET, dtype: int64

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier

In [17]:
X = df_ds.drop(columns = ['TARGET'], axis=1)
y = df_ds['TARGET']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state= 42, stratify=y)

In [ ]:
rf = RandomForestClassifier

# up-sampling 
# SMOTE